In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('Asset_price.csv')
df = df[::-1]

df.index = df['Date']
df.index = pd.to_datetime(df.index)

df = df[['Close']]
df = df.rename(columns={"Close": "s"})
df.s = df.s.apply(lambda x: x.replace(',', ''))
df.s = df.s.astype('float')

df['sigma'] = 0
df['S'] = df['s'] / 1.35
df['dS'] = df['S'] - df['S'].shift(1)
df['RollM'] = df.s.rolling(window=30,min_periods=30).mean()
df['diff'] = df['s'] - df['RollM']
df['sigma'] = np.sqrt(df['diff'] * df['diff'] / 30)

riskrate_df = pd.read_csv('Risk_rate.csv')
riskrate_df.index = pd.to_datetime(riskrate_df.Date)

riskrate_df = riskrate_df[['Close']]
riskrate_df = riskrate_df.rename(columns={"Close": "r"})

combined_df = df.join(riskrate_df, lsuffix='_caller', rsuffix='_other')
combined_df = combined_df.dropna()

combined_df.r = combined_df.r.apply(lambda x: x.replace('%', ''))
combined_df.r = combined_df.r.astype('float') / 100

combined_df['c'] = combined_df.r - 0.01


combined_df = combined_df[['s', 'sigma', 'S', 'dS', 'r', 'c']]

combined_df

,s,sigma,S,dS,r,c
Date,,,,,,
2022-06-21,3764.79,37.045945,2788.733333,66.629630,0.01642,0.00642
2022-06-22,3759.89,36.532608,2785.103704,-3.629630,0.01562,0.00562
2022-06-23,3795.73,28.739611,2811.651852,26.548148,0.01601,0.00601
2022-06-24,3911.74,7.416529,2897.585185,85.933333,0.01664,0.00664
2022-06-27,3900.11,9.357475,2888.970370,-8.614815,0.01669,0.00669
...,...,...,...,...,...,...
2024-05-01,5018.39,21.704845,3717.325926,-12.814815,0.05400,0.04400
2024-05-02,5064.20,12.364837,3751.259259,33.933333,0.05383,0.04383
2024-05-03,5127.79,0.062745,3798.362963,47.103704,0.05386,0.04386


In [20]:
f = open('../julia_scripts/julia_data.txt', "a") 
for i, row in combined_df.iterrows():
    for name, value in row.items():
        f.write(f'{name}={value}\n')
    f.write('----\n')
f.close()